# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 308 kB 4.4 MB/s 
     |████████████████████████████████| 81 kB 8.3 MB/s 
     |████████████████████████████████| 210 kB 56.5 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |████████████████████████████████| 150 kB 55.6 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 113 kB 57.5 MB/s 


In [2]:
import optuna

optuna.__version__

'2.10.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection

def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.
    
    clf = sklearn.ensemble.RandomForestClassifier(    
        n_estimators=5, max_depth=3)  # Define the model.
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()  # Train and evaluate the model.

print('Accuracy: {}'.format(objective()))

Accuracy: 0.9466666666666667


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna

def objective(trial):
    iris = sklearn.datasets.load_iris()
    
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))
    
    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2022-03-07 16:39:47,538] A new study created in memory with name: no-name-3e3a9d71-cc80-4311-8186-cc7e5aef197e
[I 2022-03-07 16:39:47,658] Trial 0 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 13, 'max_depth': 4.656141687679648}. Best is trial 0 with value: 0.9666666666666667.
[I 2022-03-07 16:39:47,801] Trial 1 finished with value: 0.96 and parameters: {'n_estimators': 12, 'max_depth': 4.54553028378252}. Best is trial 0 with value: 0.9666666666666667.
[I 2022-03-07 16:39:47,886] Trial 2 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 12, 'max_depth': 3.4440977779431448}. Best is trial 0 with value: 0.9666666666666667.
[I 2022-03-07 16:39:47,952] Trial 3 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 10, 'max_depth': 26.017873193949292}. Best is trial 0 with value: 0.9666666666666667.
[I 2022-03-07 16:39:47,998] Trial 4 finished with value: 0.94 and parameters: {'n_estimators': 6, 'max_depth': 10.709772769

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 15, 'max_depth': 3.0427512131209546}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2022-03-07 16:40:00,240] A new study created in memory with name: no-name-2a6ec8d1-b142-45d4-bcb1-d1a85aea021d
[I 2022-03-07 16:40:00,387] Trial 0 finished with value: 0.9466666666666667 and parameters: {'classifier': 'RandomForest', 'n_estimators': 13, 'max_depth': 2.939823293227295}. Best is trial 0 with value: 0.9466666666666667.
[I 2022-03-07 16:40:00,482] Trial 1 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 7, 'max_depth': 28.309332064510258}. Best is trial 1 with value: 0.96.
[I 2022-03-07 16:40:00,525] Trial 2 finished with value: 0.96 and parameters: {'classifier': 'RandomForest', 'n_estimators': 3, 'max_depth': 5.199627155942958}. Best is trial 1 with value: 0.96.
[I 2022-03-07 16:40:00,622] Trial 3 finished with value: 0.84 and parameters: {'classifier': 'RandomForest', 'n_estimators': 9, 'max_depth': 1.6218908899178455}. Best is trial 1 with value: 0.96.
[I 2022-03-07 16:40:00,802] Trial 4 finished with value: 0.9533333333333333

Accuracy: 0.9866666666666667
Best hyperparameters: {'classifier': 'SVC', 'svc_c': 3.6786412229375416}


### Plotting the study

Plotting the optimization history of the study.

In [6]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [7]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [8]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])